In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 33
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)

556949
556916
['\ufeffThe Project Gutenberg EBook of C', 'The Project Gutenberg EBook of Ch', 'he Project Gutenberg EBook of Che', 'e Project Gutenberg EBook of Ches', ' Project Gutenberg EBook of Chess']



torch.Size([556916, 33])


In [8]:
importlib.reload(autoregressive)    
net = autoregressive.TransformerEncoder(char2i, i2char)
net.to(device)
performance = net.fit(encoded, save_path = "checkpoints/autoregressive_transformer.pt")
net.save_architecture("architectures/autoregressive_transformer.architecture")

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 164,255

Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------
    1 |   1.7151 |     45.805 |         92.48
    2 |   1.2631 |     36.594 |        186.35
    3 |   1.1553 |     33.917 |        281.52
    4 |   1.0972 |     32.520 |        382.42
    5 |   1.0593 |     31.620 |        476.62


In [9]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5, :12])

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 164,255

tensor([ -9.2367,  -9.2016,  -8.1720,  -8.9351, -10.3069], device='cuda:0')


['\ufeffThe Project Gutenberg-tm the pa',
 'The Project Gutenberg-tm the paw',
 'he Project Gutenberg-tm the pawn',
 "e Project Gutenberg-tm the King'",
 ' Project Gutenberg-tm the pawn a']

In [10]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5, :12])

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 164,255

tensor([-28.9927, -30.3730, -26.7802, -23.4505, -12.8352], device='cuda:0')


['\ufeffThe Project Gutenbefforam,\nand ',
 'The Project procese, folllows to',
 'he Project Game For B6ch, which\n',
 'e Project Gutenbernal for 12. ..',
 ' Project Gutenberg-tm the suppor']

In [11]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5, :12])

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 164,255

tensor([[ -7.9266,  -8.1982,  -9.0614,  -9.2367,  -9.2722],
        [ -7.8441,  -8.8582,  -9.2016,  -9.3110,  -9.6437],
        [ -8.0777,  -9.2724,  -9.3451,  -9.4304,  -9.4867],
        [ -8.2489,  -8.2498,  -9.3699,  -9.5252, -10.1802],
        [ -9.1748,  -9.6040, -10.0753, -10.5528, -10.6235]], device='cuda:0')
[['\ufeffThe Project Gutenberg-tm of the',
  '\ufeffThe Project Gutenberg-tm 2. ...',
  '\ufeffThe Project Gutenberg-tm 2. P-K',
  '\ufeffThe Project Gutenberg-tm the pa',
  '\ufeffThe Project Gutenberg-tm 2. P-Q'],
 ['The Project Gutenberg-tm of the ',
  'The Project Gutenberg-tm on the ',
  'The Project Gutenberg-tm the paw',
  'The Project Gutenberg-tm the Kin',
  'The Project Gutenberg-tm with th'],
 ['

In [12]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)

idx, log_probabilities = net.predict(encoded[:5, :12])

net.tensor2text(idx)

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 32
Embedding dimension: 64
Feedforward dimension: 256
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 164,255



['\ufeffThe Project Gutenberg-tm of the',
 'The Project Gutenberg-tm of the ',
 'he Project Gutenberg-tm 2. ...  ',
 'e Project Gutenberg-tm with the ',
 ' Project Gutenberg-tm 2. ...    ']